In [16]:
!pip install optuna

You should consider upgrading via the '/Users/kitamurahiroto/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [17]:
#Grid search #Random search #model base

In [2]:
import optuna
optuna.__version__

'2.7.0'

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.8.1'

In [4]:
import pytorch_lightning as pl

In [5]:
def objective(trial):
    
    x = trial.suggest_uniform('x', -10,10)
    
    return(x-2)**2
    
    

In [6]:
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=10)

[I 2021-04-27 19:39:24,793] A new study created in memory with name: no-name-a042295a-7e9a-4f52-b577-7962d117ce62
[I 2021-04-27 19:39:24,795] Trial 0 finished with value: 1.0480229520791995 and parameters: {'x': 0.9762700785464951}. Best is trial 0 with value: 1.0480229520791995.
[I 2021-04-27 19:39:24,797] Trial 1 finished with value: 5.307436050111796 and parameters: {'x': 4.30378732744839}. Best is trial 0 with value: 1.0480229520791995.
[I 2021-04-27 19:39:24,798] Trial 2 finished with value: 0.0030544989253336228 and parameters: {'x': 2.055267521432878}. Best is trial 2 with value: 0.0030544989253336228.
[I 2021-04-27 19:39:24,800] Trial 3 finished with value: 1.2151454066214245 and parameters: {'x': 0.8976636599379368}. Best is trial 2 with value: 0.0030544989253336228.
[I 2021-04-27 19:39:24,801] Trial 4 finished with value: 12.439051918480782 and parameters: {'x': -1.5269040132219054}. Best is trial 2 with value: 0.0030544989253336228.
[I 2021-04-27 19:39:24,803] Trial 5 finish

In [7]:
study.best_params

{'x': 2.055267521432878}

In [8]:
###dataser breastcancer
import tensor

ModuleNotFoundError: No module named 'tensor'

In [9]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
x = breast_cancer['data']
t = breast_cancer['target']

In [10]:
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [11]:
dataset = torch.utils.data.TensorDataset(x,t)
dataset

In [12]:
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) *0.2)
n_test = len(dataset) - n_train - n_val

pl.seed_everything(0)

train, val, test = torch.utils.data.random_split(dataset,[n_train, n_val, n_test])

Global seed set to 0


In [13]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train,batch_size,shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(train,batch_size)
test_loader = torch.utils.data.DataLoader(train,batch_size)

In [14]:
##################fit


In [15]:
from pytorch_lightning.metrics.functional import accuracy

In [16]:
class Net(pl.LightningModule):
    
    def __init__(self, lr=0.01):
        super().__init__()
        
        self.lr = lr
        
        self.fc1 = nn.Linear(30,10)
        self.bn1 = nn.BatchNorm1d(10)
        self.fc2 = nn.Linear(10,2)
        
        
    def forward(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.bn1(h)
        h = self.fc2(h)
        return h 
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('train_loss', loss, on_step = False, on_epoch=True, prog_bar=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('val_loss', loss, on_step = False, on_epoch=True, prog_bar=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('test_loss', loss, on_step = False, on_epoch=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr = self.lr)
        return optimizer

In [17]:
net = Net(lr=0.001)

In [18]:
net.lr

0.001

In [19]:
from pytorch_lightning.callbacks import EarlyStopping

In [20]:
def objective(trial):
    
    lr = trial.suggest_loguniform('lr', 1e-5,1e-1)
    
    pl.seed_everything(0)
    net = Net(lr=lr)
    trainer = pl.Trainer(max_epochs=10, deterministic=True, callbacks=[EarlyStopping(monitor='val_loss')])
    trainer.fit(net, train_loader,val_loader)
    
    return trainer.callback_metrics['val_loss']

In [21]:
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=30)

[I 2021-04-27 19:39:35,962] A new study created in memory with name: no-name-3f5dbedf-ced3-464a-9eea-23e3c791101b
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 310   
1 | bn1  | BatchNorm1d | 20    
2 | fc2  | Linear      | 22    
-------------------------------------
352       Trainable params
0         Non-trainable params
352       Total params
0.001     Total estimated model params size (MB)
/Users/kitamurahiroto/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/Users/kitamurahiroto/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


[I 2021-04-27 19:39:37,263] Trial 0 finished with value: 0.5786927342414856 and parameters: {'lr': 0.0015676677195506068}. Best is trial 0 with value: 0.5786927342414856.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 310   
1 | bn1  | BatchNorm1d | 20    
2 | fc2  | Linear      | 22    
-------------------------------------
352       Trainable params
0         Non-trainable params
352       Total params
0.001     Total estimated model params size (MB)


[I 2021-04-27 19:39:38,490] Trial 1 finished with value: 0.37954509258270264 and parameters: {'lr': 0.007257005721594277}. Best is trial 0 with value: 0.5786927342414856.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 310   
1 | bn1  | BatchNorm1d | 20    
2 | fc2  | Linear      | 22    
-------------------------------------
352       Trainable params
0         Non-trainable params
352       Total params
0.001     Total estimated model params size (MB)


[I 2021-04-27 19:39:39,895] Trial 2 finished with value: 0.5160593390464783 and parameters: {'lr': 0.0025766385746135885}. Best is trial 0 with value: 0.5786927342414856.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 310   
1 | bn1  | BatchNorm1d | 20    
2 | fc2  | Linear      | 22    
-------------------------------------
352       Trainable params
0         Non-trainable params
352       Total params
0.001     Total estimated model params size (MB)


...

[I 2021-04-27 19:40:07,849] Trial 25 finished with value: 0.7914013266563416 and parameters: {'lr': 0.00013879569782200704}. Best is trial 21 with value: 0.8651501536369324.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 310   
1 | bn1  | BatchNorm1d | 20    
2 | fc2  | Linear      | 22    
-------------------------------------
352       Trainable params
0         Non-trainable params
352       Total params
0.001     Total estimated model params size (MB)


[I 2021-04-27 19:40:09,167] Trial 26 finished with value: 0.8453336954116821 and parameters: {'lr': 6.461112276678385e-05}. Best is trial 21 with value: 0.8651501536369324.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 310   
1 | bn1  | BatchNorm1d | 20    
2 | fc2  | Linear      | 22    
-------------------------------------
352       Trainable params
0         Non-trainable params
352       Total params
0.001     Total estimated model params size (MB)


[I 2021-04-27 19:40:10,240] Trial 27 finished with value: 0.8626821637153625 and parameters: {'lr': 1.7935421547486375e-05}. Best is trial 21 with value: 0.8651501536369324.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 310   
1 | bn1  | BatchNorm1d | 20    
2 | fc2  | Linear      | 22    
-------------------------------------
352       Trainable params
0         Non-trainable params
352       Total params
0.001     Total estimated model params size (MB)


[I 2021-04-27 19:40:11,371] Trial 28 finished with value: 0.3507061302661896 and parameters: {'lr': 0.017153181549546002}. Best is trial 21 with value: 0.8651501536369324.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 310   
1 | bn1  | BatchNorm1d | 20    
2 | fc2  | Linear      | 22    
-------------------------------------
352       Trainable params
0         Non-trainable params
352       Total params
0.001     Total estimated model params size (MB)


[I 2021-04-27 19:40:12,808] Trial 29 finished with value: 0.8552924394607544 and parameters: {'lr': 3.9451764772158706e-05}. Best is trial 21 with value: 0.8651501536369324.


In [22]:
study.best_trial

FrozenTrial(number=21, values=[0.8651501536369324], datetime_start=datetime.datetime(2021, 4, 27, 19, 40, 0, 811899), datetime_complete=datetime.datetime(2021, 4, 27, 19, 40, 1, 907737), params={'lr': 1.0032688516914792e-05}, distributions={'lr': LogUniformDistribution(high=0.1, low=1e-05)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=21, state=TrialState.COMPLETE, value=None)

In [23]:
study.best_params

{'lr': 1.0032688516914792e-05}

In [24]:
study.best_value

0.8651501536369324

In [29]:
!pip install plotly

     |████████████████████████████████| 13.2 MB 10.7 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=ccca5c855716498d8fa5adb759e81c067664dba9e7c219fc1b8c79a8f3150314
  Stored in directory: /Users/kitamurahiroto/Library/Caches/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying
You should consider upgrading via the '/Users/kitamurahiroto/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [35]:
import plotly

In [36]:
optuna.visualization.plot_optimization_history(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [37]:
optuna.visualization.plot_slice(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [ ]:
#############medil node

In [38]:
class Net(pl.LightningModule):
    
    def __init__(self,n_mid=10, lr=0.01):
        super().__init__()
        
        self.lr = lr
        self.n_mid = n_mid
        
        self.fc1 = nn.Linear(30,self.n_mid)
        self.bn1 = nn.BatchNorm1d(self.n_mid)
        self.fc2 = nn.Linear(self.n_mid,2)
        
        
    def forward(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.bn1(h)
        h = self.fc2(h)
        return h 
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('train_loss', loss, on_step = False, on_epoch=True, prog_bar=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('val_loss', loss, on_step = False, on_epoch=True, prog_bar=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('test_loss', loss, on_step = False, on_epoch=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr = self.lr)
        return optimizer

In [42]:
net = Net(n_mid=30, lr=0.01)

In [45]:
net.n_mid, net.lr

(30, 0.01)

In [47]:
def objective(trial):
    
    lr = trial.suggest_loguniform('lr', 1e-5,1e-1)
    
    
    
    #
    n_mid = trial.suggest_int('n_mid',2, 100)
    
    pl.seed_everything(0)
    net = Net(n_mid=n_mid, lr=lr)
    trainer = pl.Trainer(max_epochs=10, deterministic=True, callbacks=[EarlyStopping(monitor='val_loss')])
    trainer.fit(net, train_loader,val_loader)
    
    return trainer.callback_metrics['val_loss']

In [48]:
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=30)

[I 2021-04-28 08:11:11,417] A new study created in memory with name: no-name-fbab3861-732d-4467-91f0-64325e033052
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 2.2 K 
1 | bn1  | BatchNorm1d | 144   
2 | fc2  | Linear      | 146   
-------------------------------------
2.5 K     Trainable params
0         Non-trainable params
2.5 K     Total params
0.010     Total estimated model params size (MB)


[I 2021-04-28 08:11:12,752] Trial 0 finished with value: 0.34680691361427307 and parameters: {'lr': 0.0015676677195506068, 'n_mid': 72}. Best is trial 0 with value: 0.34680691361427307.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 1.7 K 
1 | bn1  | BatchNorm1d | 110   
2 | fc2  | Linear      | 112   
-------------------------------------
1.9 K     Trainable params
0         Non-trainable params
1.9 K     Total params
0.008     Total estimated model params size (MB)


[I 2021-04-28 08:11:14,041] Trial 1 finished with value: 0.3023878037929535 and parameters: {'lr': 0.0025766385746135885, 'n_mid': 55}. Best is trial 0 with value: 0.34680691361427307.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 2.0 K 
1 | bn1  | BatchNorm1d | 130   
2 | fc2  | Linear      | 132   
-------------------------------------
2.3 K     Trainable params
0         Non-trainable params
2.3 K     Total params
0.009     Total estimated model params size (MB)


[I 2021-04-28 08:11:15,349] Trial 2 finished with value: 0.5196890234947205 and parameters: {'lr': 0.0004950159553733195, 'n_mid': 65}. Best is trial 2 with value: 0.5196890234947205.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 2.8 K 
1 | bn1  | BatchNorm1d | 180   
2 | fc2  | Linear      | 182   
-------------------------------------
3.2 K     Trainable params
0         Non-trainable params
3.2 K     Total params
0.013     Total estimated model params size (MB)


[I 2021-04-28 08:11:16,572] Trial 3 finished with value: 0.45619261264801025 and parameters: {'lr': 0.0005627932047415167, 'n_mid': 90}. Best is trial 2 with value: 0.5196890234947205.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 1.2 K 
1 | bn1  | BatchNorm1d | 78    
2 | fc2  | Linear      | 80    
-------------------------------------
1.4 K     Trainable params
0         Non-trainable params
1.4 K     Total params
0.005     Total estimated model params size (MB)


...

[I 2021-04-28 08:11:46,891] Trial 25 finished with value: 0.5451490879058838 and parameters: {'lr': 2.2611640870586845e-05, 'n_mid': 44}. Best is trial 18 with value: 0.9611223340034485.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 589   
1 | bn1  | BatchNorm1d | 38    
2 | fc2  | Linear      | 40    
-------------------------------------
667       Trainable params
0         Non-trainable params
667       Total params
0.003     Total estimated model params size (MB)


[I 2021-04-28 08:11:48,383] Trial 26 finished with value: 0.6021801233291626 and parameters: {'lr': 0.0001202212505483403, 'n_mid': 19}. Best is trial 18 with value: 0.9611223340034485.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 434   
1 | bn1  | BatchNorm1d | 28    
2 | fc2  | Linear      | 30    
-------------------------------------
492       Trainable params
0         Non-trainable params
492       Total params
0.002     Total estimated model params size (MB)


[I 2021-04-28 08:11:49,853] Trial 27 finished with value: 1.186862826347351 and parameters: {'lr': 5.57586752827459e-05, 'n_mid': 14}. Best is trial 27 with value: 1.186862826347351.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 372   
1 | bn1  | BatchNorm1d | 24    
2 | fc2  | Linear      | 26    
-------------------------------------
422       Trainable params
0         Non-trainable params
422       Total params
0.002     Total estimated model params size (MB)


[I 2021-04-28 08:11:51,708] Trial 28 finished with value: 0.49710384011268616 and parameters: {'lr': 3.22830182359286e-05, 'n_mid': 12}. Best is trial 27 with value: 1.186862826347351.
Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 1.5 K 
1 | bn1  | BatchNorm1d | 94    
2 | fc2  | Linear      | 96    
-------------------------------------
1.6 K     Trainable params
0         Non-trainable params
1.6 K     Total params
0.007     Total estimated model params size (MB)


[I 2021-04-28 08:11:53,589] Trial 29 finished with value: 0.42313799262046814 and parameters: {'lr': 0.0010758642225756692, 'n_mid': 47}. Best is trial 27 with value: 1.186862826347351.


In [49]:
study.best_value

1.186862826347351

In [52]:
optuna.visualization.plot_contour(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [53]:
################
layers = nn.Sequential()
layers

Sequential()

In [54]:
fc = nn.Linear(10,10)

In [56]:
layers.add_module('fc1',fc)

In [57]:
layers

Sequential(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
)

In [60]:
layers = nn.Sequential()
for n in range(3):
    layers.add_module(f'fc{n+1}', nn.Linear(30,30))
    
    layers.add_module(f'act{n+1}', nn.ReLU())
    
    layers.add_module(f'bn{n+1}', nn.BatchNorm1d(30,30))

In [61]:
layers

Sequential(
  (fc1): Linear(in_features=30, out_features=30, bias=True)
  (act1): ReLU()
  (bn1): BatchNorm1d(30, eps=30, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=30, out_features=30, bias=True)
  (act2): ReLU()
  (bn2): BatchNorm1d(30, eps=30, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=30, out_features=30, bias=True)
  (act3): ReLU()
  (bn3): BatchNorm1d(30, eps=30, momentum=0.1, affine=True, track_running_stats=True)
)

In [62]:
y = layers(x)
y.shape

torch.Size([569, 30])

In [69]:
class Net(pl.LightningModule):
    
    def __init__(self,n_layers=1,n_mid=10, lr=0.01):
        super().__init__()
        
        self.lr = lr
        self.n_mid = n_mid
        self.n_layers = n_layers

        self.layers = nn.Sequential()
        
        self.layers = nn.Sequential()
        for n in range(self.n_layers):
            if n == 0:
                self.layers.add_module(f'fc{n+1}',nn.Linear(30,self.n_mid))
            else:
                self.layers.add_module(f'fc{n+1}',nn.Linear(self.n_mid,self.n_mid)
                                       
#             self.layers.add_module(f'act{n+1}',nn.ReLU())                                   
            self.layers.add_module(f'bn{n+1}',nn.BatchNorm1d(self.n_mid))            
        
        
        
        
        self.layers.add_module(f'fc{self.n_layers+1}',nn.Linear(self.n_mid,2))
        
        
    def forward(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.bn1(h)
        h = self.fc2(h)
        return h 
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('train_loss', loss, on_step = False, on_epoch=True, prog_bar=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('val_loss', loss, on_step = False, on_epoch=True, prog_bar=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
#         acc = accuracy(y, t)
        #log memory
        self.log('test_loss', loss, on_step = False, on_epoch=True)
#         self.log('train_loss', acc, on_epoch=True, prog_bar=True)
        return loss




    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr = self.lr)
        return optimizer

SyntaxError: invalid syntax (<ipython-input-69-12bd1ffde19b>, line 20)